In [1]:
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
from sklearn.preprocessing import StandardScaler

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import joblib

# Charger la data PCA transformée
df = pd.read_csv('Transformed data/pca_transformed_dataset_ultimate_tensile_strength.csv')

# Séparer les features et la target
X = df.drop(columns=['Ultimate tensile strength (MPa)'])
y = df['Ultimate tensile strength (MPa)']

# Séparer en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




In [3]:
# Initialiser les modèles de régression
models = {
    'LinearRegression': LinearRegression(),
    'RidgeRegression': Ridge(),
    'LassoRegression': Lasso(),
    'ElasticNetRegression': ElasticNet(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'SVR': SVR()
}


In [4]:
# Définir les grilles de recherche d'hyperparamètres pour GridSearch
param_grids = {
    'LinearRegression': {},
    'RidgeRegression': {'alpha': [0.1, 1.0, 10.0, 100.0]},
    'LassoRegression': {'alpha': [0.1, 1.0, 10.0, 100.0]},
    'ElasticNetRegression': {'alpha': [0.1, 1.0, 10.0], 'l1_ratio': [0.2, 0.5, 0.8]},
    'DecisionTreeRegressor': {'max_depth': [5, 10, 20, None], 'min_samples_split': [2, 10, 20]},
    'RandomForestRegressor': {'n_estimators': [100, 200], 'max_depth': [10, 20, None]},
    'GradientBoostingRegressor': {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 10]},
    'SVR': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']} 
}


In [5]:
# Définir les scorers
scoring = {
    'MSE': make_scorer(mean_squared_error, greater_is_better=False),
    'RMSE': make_scorer(root_mean_squared_error, greater_is_better=False),
    'MAE': make_scorer(mean_absolute_error, greater_is_better=False),
    'R2': 'r2'
}


Quelle est la meilleure métrique pour refit dans notre cas ?

Pour la prédiction de la Yield strength (MPa), une métrique comme RMSE est souvent utile car elle est directement interprétable en termes de l'unité de la variable cible (MPa), ce qui permet de comprendre facilement l'ampleur des erreurs.

RMSE pourrait être un bon choix pour refit, car il permet d'avoir une mesure intuitive de l'erreur moyenne en termes d'unités MPa. Cela facilitera l'interprétation de la performance du modèle.

In [6]:
# Dictionnaire pour stocker les meilleurs modèles et paramètres
best_models = {}

# Tableau pour stocker les résultats
results = {
    'Model': [],
    'Best Parameters': [],
    'Train MSE': [],
    'Train RMSE': [],
    'Train MAE': [],
    'Train R²': []
}


In [7]:
for model_name, model in models.items():
    print(f"Training {model_name}...")
    
    grid = GridSearchCV(estimator=model, param_grid=param_grids[model_name], scoring=scoring, refit='RMSE', cv=5, return_train_score=True)
    grid.fit(X_train, y_train)
    
    # Stocker le meilleur modèle
    best_models[model_name] = grid.best_estimator_
    
    # Enregistrer les paramètres optimaux
    results['Model'].append(model_name)
    results['Best Parameters'].append(grid.best_params_)
    
    # Prédire sur l'ensemble d'entraînement
    y_train_pred = grid.predict(X_train)
    
    # Calculer les métriques pour l'entraînement
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_rmse = root_mean_squared_error(y_train, y_train_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    
    # Stocker les résultats
    results['Train MSE'].append(train_mse)
    results['Train RMSE'].append(train_rmse)
    results['Train MAE'].append(train_mae)
    results['Train R²'].append(train_r2)

# Convertir les résultats en DataFrame
results_df = pd.DataFrame(results)

# Afficher le DataFrame des résultats
results_df

Training LinearRegression...
Training RidgeRegression...
Training LassoRegression...
Training ElasticNetRegression...
Training DecisionTreeRegressor...
Training RandomForestRegressor...
Training GradientBoostingRegressor...
Training SVR...


,Model,Best Parameters,Train MSE,Train RMSE,Train MAE,Train R²
0,LinearRegression,{},3800.804649,61.650666,47.464987,0.485960
1,RidgeRegression,{'alpha': 100.0},3870.992443,62.217300,47.841271,0.476468
2,LassoRegression,{'alpha': 0.1},3800.968286,61.651993,47.452651,0.485938
3,ElasticNetRegression,"{'alpha': 0.1, 'l1_ratio': 0.2}",3820.365833,61.809108,47.507876,0.483315
4,DecisionTreeRegressor,"{'max_depth': 20, 'min_samples_split': 20}",1073.116064,32.758450,23.507006,0.854866
5,RandomForestRegressor,"{'max_depth': 20, 'n_estimators': 100}",315.135025,17.752043,11.815796,0.957380
6,GradientBoostingRegressor,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",225.488906,15.016288,11.038942,0.969504
7,SVR,"{'C': 10, 'kernel': 'rbf'}",3569.568656,59.745867,40.341970,0.517234


In [8]:
# Sauvegarder chaque meilleur modèle dans un fichier séparé
for model_name, model in best_models.items():
    joblib.dump(model, f'Trained model/{model_name}_best_model.pkl')
